In [26]:
import os
import pandas as pd
import numpy as np
import json

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import platform
system_name = platform.system()

In [27]:
tt_flag = "train"
if tt_flag == "train":
    data_path = "../data/train/"
elif tt_flag == "test":
    data_path = "../data/test/"
    
metallographic_path = data_path + "metallographic"
hardness_path = data_path + "hardness" 

In [28]:
grade_num_list = ["1cr", "2cr", "3cr", "4cr"]
process_num_list = ["process_01", "process_02", "process_03", "process_04", "process_05", "process_06", "process_07", "process_08",
                    "process_09", "process_10", "process_11", "process_12", "process_13", "process_14", "process_15", "process_16", "process_17", "process_18"]

grade_dict = {'1cr': ['0.08', '1.00', '1.00', '0.04', '0.03', '0.06', '11.50', '86.29'],
              '2cr': ['0.16', '1.00', '1.00', '0.04', '0.03', '0.06', '13.00', '84.71'],
              '3cr': ['0.26', '1.00', '1.00', '0.04', '0.03', '0.06', '12.00', '85.61'],
              '4cr': ['0.36', '0.60', '0.80', '1.04', '0.03', '0.06', '13.00', '84.11']}
process_dict = {'process_01': ['1050', '1000', '160'],
                'process_02': ['1050', '1000', '200'],
                'process_03': ['1050', '1000', '240'],
                'process_04': ['1050', '1020', '160'],
                'process_05': ['1050', '1020', '200'],
                'process_06': ['1050', '1020', '240'],
                'process_07': ['1050', '1040', '160'],
                'process_08': ['1050', '1040', '200'],
                'process_09': ['1050', '1040', '240'],
                'process_10': ['980', '1000', '160'],
                'process_11': ['980', '1000', '200'],
                'process_12': ['980', '1000', '240'],
                'process_13': ['980', '1020', '160'],
                'process_14': ['980', '1020', '200'],
                'process_15': ['980', '1020', '240'],
                'process_16': ['980', '1040', '160'],
                'process_17': ['980', '1040', '200'],
                'process_18': ['980', '1040', '240']}

In [29]:
cut_image_list = []

def get_image_paths(folder_path):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                image_paths.append(os.path.join(root, file))  
    return image_paths


for grade_num in grade_num_list:
    for process_num in process_num_list:

        pic_cut_path = metallographic_path + "/" + grade_num + "/" + process_num + "/cut"  
        for root, dirs, files in os.walk(pic_cut_path):
            for file in files:
                if file.endswith(('.jpg', '.png', '.jpeg')): 
                    cut_image_list.append(os.path.join(root, file))  

print(len(cut_image_list))


646


In [30]:
def read_hardness_data(file_path):
    df = pd.read_excel(file_path)
    for index, row in df.iterrows():
        if row[0] == 'test times':
            return df.iloc[:index]

In [31]:
cut_data_list = []
for pic_cut_path in cut_image_list:
    path_parts = os.path.normpath(pic_cut_path).split(os.sep)
    if system_name == "Windows":
        image_path = "../../datasets" + pic_cut_path[2:]
    elif system_name == "Linux":
        image_path = "/hy-tmp/datasets" + pic_cut_path[2:]
           
    grade_value = grade_dict[path_parts[-4]]
    process_value = process_dict[path_parts[-3]]
    
    hardness_data = read_hardness_data(hardness_path + "/" + str(path_parts[-4]) + "/" + str(path_parts[-3][:-3] + path_parts[-3][-2:]) + "_1.xls")
    hardness_curve = hardness_data.iloc[:,5].tolist() 
    if len(hardness_curve) >= 14:
        hardness_curve = hardness_curve[:14]
    else:
        last_value = hardness_curve[-1]
        hardness_curve.append(last_value)
        hardness_curve.append(last_value)

    cut_data_list.append({
        "image": image_path,
        "grade": grade_value,
        "process": process_value,
        "hardness_curve": hardness_curve
    })


In [32]:
with open(data_path + "data_cut.json", 'w') as f:
    json.dump(cut_data_list, f, indent=4)